In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('../mtr-datasets/atp7d.arff')
data = pd.DataFrame(data)
data.head()

,"""'departwb'""","""'departdaytype'""","""'fetchdays'""","""'fetchdow'""","""'dtd'""","""'ALLminpA'""","""'ALLminpA'l1""","""'ALLminpA'l2""","""'ALLminpA'l7""","""'ALLavgpA'""",...,"""fdisoweekday4""","""fdisoweekday5""","""fdisoweekday6""","""fdisoweekday7""",LBL+ALLminpA+bt7d_000,LBL+ALLminp0+bt7d_000,LBL+aDLminpA+bt7d_000,LBL+aCOminpA+bt7d_000,LBL+aFLminpA+bt7d_000,LBL+aUAminpA+bt7d_000
0,72.0,1.0,60.0,3.0,12.0,359.0,372.0,351.0,317.0,615.96,...,0.0,0.0,0.0,0.0,326.0,675.0,626.0,545.0,326.0,545.0
1,79.0,1.0,69.0,5.0,10.0,336.0,298.0,298.0,320.0,616.73,...,0.0,1.0,0.0,0.0,315.0,675.0,631.0,587.0,315.0,587.0
2,79.0,1.0,70.0,6.0,9.0,336.0,336.0,298.0,320.0,628.87,...,0.0,0.0,1.0,0.0,315.0,675.0,631.0,587.0,315.0,587.0
3,79.0,1.0,71.0,7.0,8.0,336.0,336.0,336.0,330.0,632.73,...,0.0,0.0,0.0,1.0,315.0,675.0,631.0,587.0,315.0,587.0
4,79.0,1.0,72.0,1.0,7.0,336.0,336.0,336.0,330.0,635.47,...,0.0,0.0,0.0,0.0,315.0,1182.0,720.0,720.0,315.0,720.0


In [3]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values
#Making Scaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

print(X.shape,y.shape)

#Train-Test Split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=100)

(296, 411) (296, 6)


In [4]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (207, 411)
Train_Y:  (207, 6)
Test_X:  (89, 411)
Test_Y:  (89, 6)


In [5]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 296
    p = 6

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

## Bidirectional Model


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
#from keras.backend.tensorflow_backend import clear_session
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [7]:
ENS_COUNT = 8
#clear_session()

In [8]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform'))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='glorot_uniform'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model

## Forward Ensemble

In [9]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print("TRAINING ENSEMBLE {} :\n\n\n".format(j))
    #Model chain
    chain_forward = []

    #Training
    #x_t,_,y_t,_ = train_test_split(X_train,y_train,test_size=0.4)
    X_feed = X_train
    y_t = y_train
    for i in range(y_t.shape[1]):
        print("Training chain node ",i)
        y_now = y_t[:,[i,]].copy()
        print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print("Training of node {} complete\n\n".format(i))  
        #Checking accuracy of the node
        #---
        #Adding node to chain 
        chain_forward.append(node)
        #Updating X_feed
        X_feed = np.append(X_feed,y_now,axis=1)
    ensembles_fwd.append(chain_forward)

TRAINING ENSEMBLE 0 :



Training chain node  0
Shapes:
 X = (207, 411) 
 Y = (207, 1)





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
207/207 [==============================] - 0s - loss: 1.5147 - mean_squared_error: 1.5147     
Epoch 2/500
207/207 [==============================] - 0s - loss: 0.5709 - mean_squared_error: 0.5709     
Epoch 3/500
207/207 [==============================] - 0s - loss: 0.5037 - mean_squared_error: 0.5037     
Epoch 4/500
207/207 [==============================] - 0s - loss: 0.2860 - mean_squared_error: 0.2860     
Epoch 5/500
207/207 [==============================] - 0s - loss: 0.1721 - mean_squared_error: 0.1721     
Epoch 6/500
207/207 [==============================] - 0s - loss: 0.1656 - mean_squared_error: 0.1656     
Epoch 7/500
207/207 [==============================] - 0s - loss: 0.1233 - mean_squared_error: 0.1233     
Epoch 8/500
207/207 [===================

207/207 [==============================] - 0s - loss: 0.0086 - mean_squared_error: 0.0086     
Epoch 63/500
207/207 [==============================] - 0s - loss: 0.0080 - mean_squared_error: 0.0080     
Epoch 64/500
207/207 [==============================] - 0s - loss: 0.0065 - mean_squared_error: 0.0065     
Epoch 65/500
207/207 [==============================] - 0s - loss: 0.0153 - mean_squared_error: 0.0153     
Epoch 66/500
207/207 [==============================] - 0s - loss: 0.0105 - mean_squared_error: 0.0105     
Epoch 67/500
207/207 [==============================] - 0s - loss: 0.0101 - mean_squared_error: 0.0101     
Epoch 68/500
207/207 [==============================] - 0s - loss: 0.0215 - mean_squared_error: 0.0215     
Epoch 69/500
207/207 [==============================] - 0s - loss: 0.0095 - mean_squared_error: 0.0095     
Epoch 70/500
207/207 [==============================] - 0s - loss: 0.0124 - mean_squared_error: 0.0124     
Epoch 71/500
207/207 [===================

207/207 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012             
Epoch 138/500
207/207 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017             
Epoch 139/500
207/207 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012             
Epoch 140/500
207/207 [==============================] - 0s - loss: 7.4780e-04 - mean_squared_error: 7.4780e-04     
Epoch 141/500
207/207 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015             
Epoch 142/500
207/207 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016     
Epoch 143/500
207/207 [==============================] - 0s - loss: 9.3584e-04 - mean_squared_error: 9.3584e-04     
Epoch 144/500
207/207 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014     
Epoch 145/500
207/207 [==============================] - 0s - loss: 0.0019 - mean_squared_erro

207/207 [==============================] - 0s - loss: 3.5026e-05 - mean_squared_error: 3.5026e-05     
Epoch 209/500
207/207 [==============================] - 0s - loss: 3.4505e-05 - mean_squared_error: 3.4505e-05     
Epoch 210/500
207/207 [==============================] - 0s - loss: 3.2326e-05 - mean_squared_error: 3.2326e-05     
Epoch 211/500
207/207 [==============================] - 0s - loss: 2.2645e-05 - mean_squared_error: 2.2645e-05     
Epoch 212/500
207/207 [==============================] - 0s - loss: 1.9597e-05 - mean_squared_error: 1.9597e-05     
Epoch 213/500
207/207 [==============================] - 0s - loss: 1.4125e-05 - mean_squared_error: 1.4125e-05     
Epoch 214/500
207/207 [==============================] - 0s - loss: 1.1495e-05 - mean_squared_error: 1.1495e-05     
Epoch 215/500
207/207 [==============================] - 0s - loss: 1.1094e-05 - mean_squared_error: 1.1094e-05     
Epoch 216/500
207/207 [==============================] - 0s - loss: 1.7868e-05

207/207 [==============================] - 0s - loss: 0.0392 - mean_squared_error: 0.0392     
Epoch 279/500
207/207 [==============================] - 0s - loss: 0.0466 - mean_squared_error: 0.0466     
Epoch 280/500
207/207 [==============================] - 0s - loss: 0.0515 - mean_squared_error: 0.0515     
Epoch 281/500
207/207 [==============================] - 0s - loss: 0.0352 - mean_squared_error: 0.0352     
Epoch 282/500
207/207 [==============================] - 0s - loss: 0.0209 - mean_squared_error: 0.0209     
Epoch 283/500
207/207 [==============================] - ETA: 0s - loss: 0.0236 - mean_squared_error: 0.02 - 0s - loss: 0.0248 - mean_squared_error: 0.0248     
Epoch 284/500
207/207 [==============================] - 0s - loss: 0.0166 - mean_squared_error: 0.0166     
Epoch 285/500
207/207 [==============================] - 0s - loss: 0.0278 - mean_squared_error: 0.0278     
Epoch 286/500
207/207 [==============================] - 0s - loss: 0.0186 - mean_squared_

...

207/207 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018     
Epoch 207/500
207/207 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017             
Epoch 208/500
207/207 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 209/500
207/207 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023     
Epoch 210/500
207/207 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015             
Epoch 211/500
207/207 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014     
Epoch 212/500
207/207 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023     
Epoch 213/500
207/207 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 214/500
207/207 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 215/500
207

207/207 [==============================] - 0s - loss: 0.0060 - mean_squared_error: 0.0060     
Epoch 281/500
207/207 [==============================] - 0s - loss: 0.0087 - mean_squared_error: 0.0087     
Epoch 282/500
207/207 [==============================] - 0s - loss: 0.0067 - mean_squared_error: 0.0067     
Epoch 283/500
207/207 [==============================] - 0s - loss: 0.0042 - mean_squared_error: 0.0042     
Epoch 284/500
207/207 [==============================] - 0s - loss: 0.0056 - mean_squared_error: 0.0056     
Epoch 285/500
207/207 [==============================] - 0s - loss: 0.0042 - mean_squared_error: 0.0042     
Epoch 286/500
207/207 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034     
Epoch 287/500
207/207 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 288/500
207/207 [==============================] - 0s - loss: 0.0046 - mean_squared_error: 0.0046     
Epoch 289/500
207/207 [==========

207/207 [==============================] - 0s - loss: 6.0213e-04 - mean_squared_error: 6.0213e-04
Epoch 355/500
207/207 [==============================] - 0s - loss: 4.2630e-04 - mean_squared_error: 4.2630e-04     
Epoch 356/500
207/207 [==============================] - 0s - loss: 3.0756e-04 - mean_squared_error: 3.0756e-04     
Epoch 357/500
207/207 [==============================] - 0s - loss: 3.0756e-04 - mean_squared_error: 3.0756e-04     
Epoch 358/500
207/207 [==============================] - 0s - loss: 3.2261e-04 - mean_squared_error: 3.2261e-04     
Epoch 359/500
207/207 [==============================] - 0s - loss: 2.6344e-04 - mean_squared_error: 2.6344e-04     
Epoch 360/500
207/207 [==============================] - 0s - loss: 1.4635e-04 - mean_squared_error: 1.4635e-04     
Epoch 361/500
207/207 [==============================] - 0s - loss: 1.6442e-04 - mean_squared_error: 1.6442e-04     
Epoch 362/500
207/207 [==============================] - 0s - loss: 1.7247e-04 - me

207/207 [==============================] - 0s - loss: 0.0050 - mean_squared_error: 0.0050     
Epoch 426/500
207/207 [==============================] - 0s - loss: 0.0062 - mean_squared_error: 0.0062     
Epoch 427/500
207/207 [==============================] - 0s - loss: 0.0046 - mean_squared_error: 0.0046     
Epoch 428/500
207/207 [==============================] - 0s - loss: 0.0048 - mean_squared_error: 0.0048     
Epoch 429/500
207/207 [==============================] - 0s - loss: 0.0053 - mean_squared_error: 0.0053     
Epoch 430/500
207/207 [==============================] - 0s - loss: 0.0042 - mean_squared_error: 0.0042     
Epoch 431/500
207/207 [==============================] - 0s - loss: 0.0042 - mean_squared_error: 0.0042     
Epoch 432/500
207/207 [==============================] - 0s - loss: 0.0032 - mean_squared_error: 0.0032     
Epoch 433/500
207/207 [==============================] - 0s - loss: 0.0035 - mean_squared_error: 0.0035     
Epoch 434/500
207/207 [==========

207/207 [==============================] - 0s - loss: 1.6379e-04 - mean_squared_error: 1.6379e-04     
Epoch 500/500
207/207 [==============================] - 0s - loss: 1.8206e-04 - mean_squared_error: 1.8206e-04     
Training of node 5 complete




## Backward  Ensemble

In [ ]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print("TRAINING ENSEMBLE {} :\n\n\n".format(j))
    #Model chain
    chain_reverse = []
    #Training
    #x_t,_,y_t,_ = train_test_split(X_train,y_train,test_size=0.4)
    X_feed = X_train
    y_t = y_train
    for i in range(1,y_t.shape[1]+1):
        print("Training chain node ",i)
        y_now = y_t[:,[-i,]].copy()
        print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print("Training of node {} complete\n\n".format(i))  
        #Checking accuracy of the node
        #---
        #Adding node to chain 
        chain_reverse.append(node)
        #Updating X_feed
        X_feed = np.append(X_feed,y_now,axis=1)
    ensembles_rev.append(chain_reverse)

TRAINING ENSEMBLE 0 :



Training chain node  1
Shapes:
 X = (207, 411) 
 Y = (207, 1)
Epoch 1/500
207/207 [==============================] - 0s - loss: 1.7785 - mean_squared_error: 1.7785     
Epoch 2/500
207/207 [==============================] - 0s - loss: 0.5194 - mean_squared_error: 0.5194     
Epoch 3/500
207/207 [==============================] - 0s - loss: 0.2833 - mean_squared_error: 0.2833     
Epoch 4/500
207/207 [==============================] - 0s - loss: 0.2788 - mean_squared_error: 0.2788     
Epoch 5/500
207/207 [==============================] - 0s - loss: 0.2537 - mean_squared_error: 0.2537     
Epoch 6/500
207/207 [==============================] - 0s - loss: 0.2384 - mean_squared_error: 0.2384     
Epoch 7/500
207/207 [==============================] - 0s - loss: 0.1427 - mean_squared_error: 0.1427     
Epoch 8/500
207/207 [==============================] - 0s - loss: 0.1377 - mean_squared_error: 0.1377     
Epoch 9/500
207/207 [==============================] - 0s

207/207 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 74/500
207/207 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014             
Epoch 75/500
207/207 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 76/500
207/207 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 77/500
207/207 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019             
Epoch 78/500
207/207 [==============================] - 0s - loss: 9.9702e-04 - mean_squared_error: 9.9702e-04
Epoch 79/500
207/207 [==============================] - ETA: 0s - loss: 0.0011 - mean_squared_error: 0.00 - 0s - loss: 0.0010 - mean_squared_error: 0.0010     
Epoch 80/500
207/207 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013             
Epoch 81/500
207/207 [==============================] - 0s - loss: 0.0

207/207 [==============================] - 0s - loss: 0.0185 - mean_squared_error: 0.0185     
Epoch 147/500
207/207 [==============================] - 0s - loss: 0.0122 - mean_squared_error: 0.0122     
Epoch 148/500
207/207 [==============================] - 0s - loss: 0.0101 - mean_squared_error: 0.0101     
Epoch 149/500
207/207 [==============================] - 0s - loss: 0.0095 - mean_squared_error: 0.0095     
Epoch 150/500
207/207 [==============================] - 0s - loss: 0.0066 - mean_squared_error: 0.0066     
Epoch 151/500
207/207 [==============================] - 0s - loss: 0.0055 - mean_squared_error: 0.0055     
Epoch 152/500
207/207 [==============================] - 0s - loss: 0.0048 - mean_squared_error: 0.0048     
Epoch 153/500
207/207 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034     
Epoch 154/500
207/207 [==============================] - 0s - loss: 0.0025 - mean_squared_error: 0.0025     
Epoch 155/500
207/207 [==========

207/207 [==============================] - 0s - loss: 1.7395e-06 - mean_squared_error: 1.7395e-06     
Epoch 217/500
207/207 [==============================] - 0s - loss: 1.5202e-06 - mean_squared_error: 1.5202e-06     
Epoch 218/500
207/207 [==============================] - 0s - loss: 1.4668e-06 - mean_squared_error: 1.4668e-06     
Epoch 219/500
207/207 [==============================] - 0s - loss: 1.5987e-06 - mean_squared_error: 1.5987e-06     
Epoch 220/500
207/207 [==============================] - 0s - loss: 1.3537e-06 - mean_squared_error: 1.3537e-06     
Epoch 221/500
207/207 [==============================] - 0s - loss: 1.1254e-06 - mean_squared_error: 1.1254e-06     
Epoch 222/500
207/207 [==============================] - 0s - loss: 2.7054e-06 - mean_squared_error: 2.7054e-06     
Epoch 223/500
207/207 [==============================] - 0s - loss: 1.5758e-06 - mean_squared_error: 1.5758e-06     
Epoch 224/500
207/207 [==============================] - 0s - loss: 1.6544e-06

207/207 [==============================] - 0s - loss: 7.0537e-07 - mean_squared_error: 7.0537e-07     
Epoch 287/500
207/207 [==============================] - 0s - loss: 4.3864e-07 - mean_squared_error: 4.3864e-07     
Epoch 288/500
207/207 [==============================] - 0s - loss: 2.7347e-07 - mean_squared_error: 2.7347e-07     
Epoch 289/500
207/207 [==============================] - 0s - loss: 6.7873e-07 - mean_squared_error: 6.7873e-07     
Epoch 290/500
207/207 [==============================] - 0s - loss: 7.6497e-07 - mean_squared_error: 7.6497e-07     
Epoch 291/500
207/207 [==============================] - 0s - loss: 1.7712e-06 - mean_squared_error: 1.7712e-06     
Epoch 292/500
207/207 [==============================] - 0s - loss: 1.3480e-05 - mean_squared_error: 1.3480e-05     
Epoch 293/500
207/207 [==============================] - 0s - loss: 1.5936e-05 - mean_squared_error: 1.5936e-05     
Epoch 294/500
207/207 [==============================] - 0s - loss: 1.4754e-05

...

207/207 [==============================] - 0s - loss: 0.0042 - mean_squared_error: 0.0042     
Epoch 211/500
207/207 [==============================] - 0s - loss: 0.0028 - mean_squared_error: 0.0028     
Epoch 212/500
207/207 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034     
Epoch 213/500
207/207 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030     
Epoch 214/500
207/207 [==============================] - 0s - loss: 0.0026 - mean_squared_error: 0.0026     
Epoch 215/500
207/207 [==============================] - 0s - loss: 0.0026 - mean_squared_error: 0.0026     
Epoch 216/500
207/207 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023     
Epoch 217/500
207/207 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 218/500
207/207 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020             
Epoch 219/500
207/207 [==

207/207 [==============================] - 0s - loss: 1.9200e-05 - mean_squared_error: 1.9200e-05     
Epoch 282/500
207/207 [==============================] - 0s - loss: 1.4807e-05 - mean_squared_error: 1.4807e-05     
Epoch 283/500
207/207 [==============================] - 0s - loss: 1.2166e-05 - mean_squared_error: 1.2166e-05     
Epoch 284/500
207/207 [==============================] - 0s - loss: 1.4796e-05 - mean_squared_error: 1.4796e-05     
Epoch 285/500
207/207 [==============================] - 0s - loss: 1.1970e-05 - mean_squared_error: 1.1970e-05     
Epoch 286/500
207/207 [==============================] - 0s - loss: 1.0856e-05 - mean_squared_error: 1.0856e-05     
Epoch 287/500
207/207 [==============================] - 0s - loss: 1.0027e-05 - mean_squared_error: 1.0027e-05     
Epoch 288/500
207/207 [==============================] - 0s - loss: 1.0930e-05 - mean_squared_error: 1.0930e-05     
Epoch 289/500
207/207 [==============================] - 0s - loss: 1.0075e-05

207/207 [==============================] - 0s - loss: 8.8306e-05 - mean_squared_error: 8.8306e-05     
Epoch 352/500
207/207 [==============================] - 0s - loss: 1.3450e-04 - mean_squared_error: 1.3450e-04     
Epoch 353/500
207/207 [==============================] - 0s - loss: 2.3281e-04 - mean_squared_error: 2.3281e-04     
Epoch 354/500
207/207 [==============================] - 0s - loss: 1.1158e-04 - mean_squared_error: 1.1158e-04     
Epoch 355/500
207/207 [==============================] - 0s - loss: 3.0272e-04 - mean_squared_error: 3.0272e-04     
Epoch 356/500
207/207 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017             
Epoch 357/500
207/207 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022     
Epoch 358/500
207/207 [==============================] - 0s - loss: 8.8182e-04 - mean_squared_error: 8.8182e-04
Epoch 359/500
207/207 [==============================] - 0s - loss: 6.9414e-04 - mean_squar

207/207 [==============================] - 0s - loss: 2.5385e-04 - mean_squared_error: 2.5385e-04     
Epoch 424/500
207/207 [==============================] - 0s - loss: 9.7049e-04 - mean_squared_error: 9.7049e-04     
Epoch 425/500
207/207 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018             
Epoch 426/500
207/207 [==============================] - 0s - loss: 7.4443e-04 - mean_squared_error: 7.4443e-04
Epoch 427/500
207/207 [==============================] - 0s - loss: 4.7302e-04 - mean_squared_error: 4.7302e-04     
Epoch 428/500
207/207 [==============================] - 0s - loss: 5.0176e-04 - mean_squared_error: 5.0176e-04     
Epoch 429/500
207/207 [==============================] - 0s - loss: 2.8343e-04 - mean_squared_error: 2.8343e-04     
Epoch 430/500
207/207 [==============================] - 0s - loss: 2.9783e-04 - mean_squared_error: 2.9783e-04     
Epoch 431/500
207/207 [==============================] - 0s - loss: 3.4149e-04 - me

207/207 [==============================] - 0s - loss: 0.0036 - mean_squared_error: 0.0036     
Epoch 497/500
207/207 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 498/500
207/207 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034     
Epoch 499/500
207/207 [==============================] - 0s - loss: 0.0040 - mean_squared_error: 0.0040     
Epoch 500/500
207/207 [==============================] - 0s - loss: 0.0044 - mean_squared_error: 0.0044     
Training of node 4 complete


Training chain node  5
Shapes:
 X = (207, 415) 
 Y = (207, 1)
Epoch 1/500


## Mean Aggregation

In [ ]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
            

In [ ]:
##Calculating the mean
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [ ]:
#Adding all results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2))

### Evaluating Mean Aggregation

In [ ]:
evaluate(y_scaler.inverse_transform(y_test),
         y_scaler.inverse_transform(np.array(predictions)))